In [1]:
from keras import Model
from keras import layers
from matplotlib import pyplot
from keras.datasets import cifar100
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

def resnet_block(x, filters, stride = (1,1), match = False):

  shortcut = x
  x = layers.Conv2D(filters, (1,1), strides = stride)(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)

  x = layers.Conv2D(filters, (3,3), padding = 'same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.ReLU()(x)

  x = layers.Conv2D(filters*4, (1,1))(x)
  x = layers.BatchNormalization()(x)

  if (match == True):#To match size of shortcut to the output if needed
    shortcut = layers.Conv2D(filters*4, (1,1), strides = stride)(shortcut)
  
  x = layers.add([x,shortcut])
  x = layers.ReLU()(x)

  return x

In [2]:
def load_dataset():
	# load dataset
	(x_train, y_train), (x_test, y_test) = cifar100.load_data()
	# one hot encode target values
	y_train = to_categorical(y_train)
	y_test = to_categorical(y_test)
	return x_train, y_train, x_test, y_test

In [3]:
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [4]:
x_train, y_train, x_test, y_test = load_dataset()
x_train, x_test = prep_pixels(x_train, x_test)


169009152/169001437 [==============================] - 4s 0us/step


In [5]:
inputs = layers.Input(shape = (32,32,3))
x = layers.Conv2D(64, kernel_size=(3,3), padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)


x = resnet_block(x, 64, match = True)

for _ in range(2):
  x = resnet_block(x,64)

#double the number of filters
x = resnet_block(x, 128, stride = (2,2), match = True)

for _ in range(3):
  x = resnet_block(x,128)

#double the number of filters
x = resnet_block(x, 256, stride = (2,2), match = True)

for _ in range(5):
  x = resnet_block(x,256)

x = resnet_block(x, 512, stride = (2,2), match = True)

for _ in range ( 2 ):
  x = resnet_block(x, 512)

x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(100, activation = 'softmax')(x)

model = Model(inputs,outputs )
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [8]:
from keras.callbacks import ModelCheckpoint
filepath='/content/sample_data/best.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history = model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_test, y_test), callbacks=callbacks_list)
model.save('cifar model')

Epoch 1/50
782/782 [==============================] - 161s 206ms/step - loss: 4.1186 - accuracy: 0.0818 - val_loss: 6.4383 - val_accuracy: 0.0973
Epoch 2/50
782/782 [==============================] - 162s 207ms/step - loss: 3.3497 - accuracy: 0.1888 - val_loss: 3.4001 - val_accuracy: 0.2122
Epoch 3/50
782/782 [==============================] - 162s 207ms/step - loss: 2.8567 - accuracy: 0.2830 - val_loss: 3.0806 - val_accuracy: 0.2334
Epoch 4/50
782/782 [==============================] - 162s 207ms/step - loss: 2.6489 - accuracy: 0.3309 - val_loss: 4.6786 - val_accuracy: 0.3031
Epoch 5/50
782/782 [==============================] - 161s 206ms/step - loss: 2.3379 - accuracy: 0.3884 - val_loss: 2.4796 - val_accuracy: 0.3704
Epoch 6/50
782/782 [==============================] - 161s 206ms/step - loss: 1.9673 - accuracy: 0.4676 - val_loss: 2.9457 - val_accuracy: 0.4303
Epoch 7/50
782/782 [==============================] - 161s 206ms/step - loss: 1.7153 - accuracy: 0.5216 - val_loss: 2.1387 -